In [ ]:
import requests
import string
import json
import re
import pandas as pd


In [ ]:
filename = 'APIcall.csv'

#Making file for caching and working on dataset, do not need to retain all the data
write_column_names = ['title', 'author', 'published_date', 'link', 'excerpt', 'summary', 'rank', 'topic', 'clean_url']

#Making file to save permanently to our database


In [ ]:
endpoint = 'https://api.newscatcherapi.com/v2/search?'
places = ['Boulder', 'Longmont']
start_date = '2024/01/20'
end_date = '2024/02/05'
headers = {'x-api-key': 'CmKYSqkNfns_AIfY0Rs9AmxcSYJhf6nVuFchtXKsOr8'}

#Add constraint using regex to avoid any searches with Reporting by Brad Brooks in Longmont, Colorado

params = {
    'q': 'Longmont, Colorado',
    'lang': 'en',
    'countries': 'US',
    'ranked_only': True,
    'sort_by': 'rank',
    'page_size': 100,
    'page':1,
    'to': end_date,
    'from': start_date
}

response = requests.get(endpoint, headers=headers, params=params)
print(response)
json_text = response.json()
json_str = json.dumps(json_text, indent=4)
print(json_str)